# Google Authentication

In [1]:
import sys
if 'google.colab' in sys.modules:
    #!pip install langchain google-cloud-aiplatform rich
    google_auth.authenticate_user()

In [2]:

import os
import requests
from dotenv import load_dotenv

load_dotenv()

True

In [3]:

from langchain.llms import VertexAI
from langchain.agents import AgentType, ZeroShotAgent, Tool, AgentExecutor, initialize_agent, tool
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain


In [4]:

from google.cloud import aiplatform

#! Change the following to your own project and location
aiplatform.init(project="aerobic-gantry-387923", location="us-central1")
vertex = VertexAI(
    temperature=0.2,
    max_tokens=1024,
    top_p=0.9,
    top_k=40
    
)


In [19]:


import Tools
agent_tools = Tools.Tools()

tools = [
    
    Tool(
        name = "Degree Search",
        func = agent_tools.get_degrees_from_query,
        description = "This tool allows you to search for UT Dallas' Degrees, Majors, Minors, and Certificates using keywords. It returns multiple titles, snippet and URLs that you can use extract text tool on.",
    ),
    Tool(
        name = "Course Search",
        func = agent_tools.get_courses_from_query,
        description = "This tool enables you to search for UT Dallas' courses using keywords. Do not put anyone name including the professor at input. Use this tool to compare multiple courses. It returns multiple titles, snippet and URLs that you can use extract text tool on.",
    ),

    Tool(
        name = "General Search",
        func = agent_tools.utdallas_search,
        description = "Please use this tool only as a last resort. It performs a comprehensive search across all UT Dallas pages that will not give information about courses or degrees. It returns multipl possible titles, snippet and URLs that you can use extract text tool on.",
    ),
    Tool(
        name = "Extract Text from UT Dallas Site and understand the content",
        func = agent_tools.access_utdallas_site,
        description = "If you have a URL string containing utdallas.edu. Pass in that link and the user question to use this function for wanted data.",
    ),
    Tool(
        name = "Dictionary",
        func = agent_tools.dictionary_search,
        description = "If you need the definition of a word or phrase unrelated to UT Dallas, use this function to retrieve the definition. Do not use for anything related to UT Dallas only information.",
    )
]


In [20]:
agent_tools.get_courses_from_query("Dr. Gopal Gupta")

'No results found.'

In [21]:

prefix = """
You should answer the following questions with up-to-date information. You have access to the following tools:"""

format_instruction = """You must use the following format for all responses or your response will be considered incorrect:

Question: the input question you must answer

Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action in a single sentence
... (this Thought/Action/Action Input/Observation can repeat N times but Question should only appear once)
Thought: I now know the final answer
Final Answer: the final answer to the original input question.
If you do not know the answer, you can say so.
"""

suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}

"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    format_instructions=format_instruction,
    input_variables=["input", "chat_history", "agent_scratchpad"],

)

In [22]:

memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(llm=vertex, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,
    handle_parsing_errors=False,
)


In [29]:
try:
    response = agent_chain.run(input="What is the difficulty of ACCT 2301 compared to ACCT 2302?")
except ValueError as e:
    print("Error :()")
    response = str(e)
    prefix = "Could not parse LLM output: `"
    if not response.startswith("Could not parse LLM output: `"):
        raise e
    response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")

print(response)



> Entering new  chain...
Thought: I need to know the difficulty of ACCT 2301 and ACCT 2302
Action: Course Search
Action Input: ACCT 2301
Observation: {'items': [{'title': 'Introductory Financial Accounting - UT Dallas 2018 Undergraduate ...', 'link': 'https://catalog.utdallas.edu/2018/undergraduate/courses/acct2301', 'snippet': 'ACCT2301 - Introductory Financial Accounting. ACCT 2301 (ACCT 2301) Introductory Financial Accounting (3 semester credit hours) An introduction to financial\xa0...'}, {'title': 'Accounting - UT Dallas 2020 Undergraduate Catalog - The ...', 'link': 'https://catalog.utdallas.edu/2020/undergraduate/courses/acct', 'snippet': 'ACCT 2301 (ACCT 2301) Introductory Financial Accounting (3 semester credit hours) An introduction to financial reporting designed to create an awareness of\xa0...'}, {'title': 'Introductory Financial Accounting - UT Dallas 2022 Undergraduate ...', 'link': 'https://catalog.utdallas.edu/2022/undergraduate/courses/acct2301', 'snippet': 'ACCT230

In [10]:
print(agent_chain.memory.buffer)